In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


CALLBACKS




In [ ]:
from tensorflow.keras.callbacks import Callback
import pickle
from datetime import datetime
import time

class MyCheckpoint(Callback):
  def __init__(self, start_epoch, exp_folder, weights_path, pickles_path, EXP_NAME, model_name):
    super(MyCheckpoint, self).__init__()
    self.current_epoch = start_epoch + 1  
    self.timestamps = {}
    self.exp_folder = exp_folder
    self.weights_path = weights_path
    self.pickles_path = pickles_path
    self.EXP_NAME = EXP_NAME  
    self.model_name = model_name
    self.model_training_time = 0
    self.start_batch = 0
    self.end_batch = 0


  def on_epoch_begin(self, epoch, logs=None):
    self.timestamps[self.current_epoch] = {}   
    self.timestamps[self.current_epoch]["start"] = datetime.now().time().strftime("%H:%M")  

  def on_test_begin(self, logs=None):    
    self.model.save_weights(os.path.join(self.weights_path+'{}_epoch_{}.h5'.format(self.model_name,self.current_epoch)))          
    self.timestamps[self.current_epoch]["end"] = datetime.now().time().strftime("%H:%M")  # PASAR ARRIBA POR EL SAVE
    self.timestamps[self.current_epoch]["train_time"] = self.model_training_time 
    with open(os.path.join(self.exp_folder, f"timestamps_epoch{self.current_epoch}.pickle"), 'wb') as fd:
      pickle.dump(self.timestamps, fd)
    self.model_training_time = 0

  def on_train_batch_begin(self, batch, logs=None):
      self.start_batch = time.time()

  def on_train_batch_end(self, batch, logs=None):
      self.end_batch = time.time()
      self.model_training_time+= (self.end_batch - self.start_batch)

  def on_epoch_end(self, epoch, logs=None):            
    with open(os.path.join(self.pickles_path+'{}_epoch_{}.pickle'.format(self.model_name,self.current_epoch)), 'wb') as fd:
      pickle.dump(logs, fd)
    self.current_epoch = self.current_epoch + 1    

In [ ]:
FOLD_NAME = '2fold'
DATASET = 'Multisampling32'
MODEL_NAME = 'rgb128_'+FOLD_NAME

In [ ]:
import os

from tensorflow.keras.initializers import HeNormal, GlorotNormal
from tensorflow.keras.regularizers import l1_l2, l1
from tensorflow.keras.activations import elu
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

basePath = "/content/gdrive/My Drive/Informática/Tesina/"

datasets = {            
            "Multisampling32":{"path": basePath+"Videos/Multisampling32/","num_frames": 32,"files_posfix":"Multisampling32"},  
            "Multisampling16":{"path": basePath+"Videos/Multisampling16/","num_frames": 16, "files_posfix":"Multisampling16"},  
            "Multisampling8":{"path": basePath+"Videos/Multisampling8/","num_frames": 8, "files_posfix":"Multisampling8"},  
            "SimpleSampling32":{"path": basePath+"Videos/SimpleSampling32/","num_frames": 32, "files_posfix":"SimpleSampling32"},              
  }


class Experiment():

  def __init__(self, dataset, model_name, fold):
    #Logging and checkpoints
    self.dataset = dataset
    self.model_name = model_name
    self.exp_folder = os.path.join(basePath+"Experimentos/MobileNet+Conv", dataset, model_name)
    self.weights_path = os.path.join(self.exp_folder,"Weights/")
    self.pickles_path = os.path.join(self.exp_folder,"Pickles/")
    if (os.path.exists(self.exp_folder)):           
      weights = os.listdir(self.weights_path)
      # get the latest epoch trained
      try:
        self.current_epoch = sorted(list(map(lambda x: int(x.split('_')[-1].split('.')[0]), weights)), reverse=True)[0] 
      except (TypeError, IndexError):
        self.current_epoch = 0      
    else:      
      os.mkdir(self.exp_folder)      
      os.mkdir(self.weights_path)
      os.mkdir(self.pickles_path)
      self.current_epoch = 0

    tensorboard = TensorBoard(log_dir=os.path.join(self.exp_folder, 'logs'), histogram_freq=1)
    early = EarlyStopping(monitor='val_accuracy', patience=4)
    self.testing = False
    #Data specs
    data_dict = datasets[dataset]
    self.fold_name = fold
    self.files_posfix = data_dict['files_posfix']
    self.names_path = os.path.join(basePath+"/DatasetsNames", data_dict['files_posfix'])
    self.videos_path = data_dict['path'] 
    self.num_frames = data_dict['num_frames']
    self.height = 128 #data_dict['height']
    self.width = 128 #data_dict['width']
    self.channels = 3
    #Model specs
    self.activation = elu
    self.initializer = GlorotNormal
    self.regularizer = None#l1(1e-4)

    self.layers = [
                        {'units':128},
                        {'units':128},
                        
                    ]

    self.dense_layer = 128
    self.batch_size = 32
    self.epochs_to_train = 48 - self.current_epoch
    self.lr = 0.001
    self.callbacks = [MyCheckpoint(self.current_epoch, self.exp_folder, self.weights_path, self.pickles_path, self.dataset, self.model_name), tensorboard, early]

  def save_resume(self):
    with open(os.path.join(self.exp_folder,'exp_resume.txt'), 'w') as f:
      stri = ""
      settings = [a for a in dir(experiment) if not a.startswith('__') and not callable(getattr(experiment, a))]
      for setting in settings:
        stri = stri + setting+": "+str(getattr(experiment, setting))+'\n'
      f.writelines(stri)


INICIALIZACION

MODELOS


In [ ]:

from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed
from tensorflow.keras import Model
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.regularizers import l2



def MobileNet_model(experiment,categories=64):
  inp = Input(shape=(None, experiment.width, experiment.height, 3))
  base_model = MobileNet(input_shape=(experiment.width, experiment.height, 3),
                                  include_top=False,
                                  weights='imagenet',
                                  input_tensor=None,
                                  pooling='avg',)
  for layer in base_model.layers:
    layer.trainable = False
  x = TimeDistributed(base_model)(inp)
  for i in range(len(experiment.layers)-1):
    x = GRU(units=experiment.layers[i]['units'], 
            return_sequences=True, 
            activation=experiment.activation, 
            kernel_initializer=experiment.initializer(), 
            kernel_regularizer=experiment.regularizer)(x)
  x = GRU(units=experiment.layers[-1]['units'], 
          return_sequences=False, 
          activation=experiment.activation,
          kernel_initializer=experiment.initializer(), 
          kernel_regularizer=experiment.regularizer)(x)  
  x = Dense(experiment.dense_layer, activation=experiment.activation, kernel_initializer=experiment.initializer(), kernel_regularizer=experiment.regularizer)(x)
  x = Dense(categories, activation='softmax')(x)
  model = Model(inputs=inp, outputs=x,name=experiment.model_name)
  return model

GENERADOR

In [ ]:
import cv2
import imageio
import numpy as np
from tensorflow.keras.utils import to_categorical
def generator_cropped(videos_path,samples, batch_size, categories=64):
  """
    samples serian los nombres de archivos
  """
  height_upper_limit = int(1080/4)
  width_upper_limit = int(1920/4)
  height_to_crop = int(height_upper_limit*3)
  width_to_crop = int(width_upper_limit*3)
  w = experiment.width
  h = experiment.height
  num_samples = len(samples) 
  while True:
    for offset in range(0,num_samples, batch_size):
      batch_samples = samples[offset:offset+batch_size]
      X_train = []
      y_train = []
      for batch_sample in batch_samples:
        fname, w_limit, h_limit = batch_sample.split(" ")
        w_limit, h_limit = int(w_limit), int(h_limit)
        vidcap = cv2.VideoCapture(videos_path+fname)      
        #vidcap = imageio.get_reader(videos_path+batch_sample)
        
        frames = []
        suc, frame = vidcap.read()                
        #for i, frame in enumerate(vidcap):
        while suc: 
          frame = frame[h_limit:h_limit+height_to_crop,w_limit:w_limit+width_to_crop,:]
          frame = cv2.resize(frame, (w,h), interpolation=cv2.INTER_AREA)
          frame =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) if (experiment.channels==3) else np.expand_dims(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),2)
          frame = frame.astype(np.float32)
          frame /= 255.0          

          frames.append(frame)
          suc, frame = vidcap.read()  
             
        X_train.append(frames)        
        cat = int(batch_sample.split('_')[0])#  Los mp4 estan categorizados de 1 a 64 y los .avi de 0 a 64, asi que no necesito restar 1        
        y_train.append(to_categorical(cat, num_classes=categories))      
      try:
        X_train = np.asarray(X_train, dtype=np.float32)      
      except:
        print("Trying again...")
        try:
          X_train = np.asarray(X_train, dtype=np.float32)
        except:
          print("failed reading")      
      y_train = np.asarray(y_train)      
      yield X_train, y_train

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import mobilenet
def generator(videos_path,samples, batch_size, categories=64):
  """
    samples serian los nombres de archivos
  """
  w = experiment.width
  h = experiment.height
  num_samples = len(samples) 
  while True:
    for offset in range(0,num_samples, batch_size):
      batch_samples = samples[offset:offset+batch_size]      
      X_train = []
      y_train = []
      for batch_sample in batch_samples:
        vidcap = cv2.VideoCapture(os.path.join(videos_path,batch_sample))              
        frames = []
        suc, frame = vidcap.read()              
        while suc: 
          frame = cv2.resize(frame, (w,h), interpolation=cv2.INTER_AREA)
          frame = mobilenet.preprocess_input(frame)   
          frames.append(frame)
          suc, frame = vidcap.read()
        
        X_train.append(frames)        
        cat = int(batch_sample.split('_')[0])#  Los mp4 estan categorizados de 1 a 64 y los .avi de 0 a 64, asi que no necesito restar 1        
        y_train.append(to_categorical(cat, num_classes=categories))      
      
      try: 
        X_train = np.asarray(X_train, dtype=np.float32)      
      except:
        print("frames len: "+str(len(frames)))
        try:
          print("Intentando de vuelta")
          X_train = np.asarray(X_train, dtype=np.float32)      
        except:
          print(len(X_train))
        print("Long de video: "+str(vidcap.get(7)))     
      y_train = np.asarray(y_train)      
      yield X_train, y_train


In [ ]:
import cv2
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.utils import Sequence
import math 
class SequenceGenerator(Sequence):
  def __init__(self, videos_path, samples,batch_size, width, height, channels):
    self.batch_size = batch_size
    self.videos_path = videos_path
    self.samples = samples
    self.width = width
    self.height = height
    self.channels = channels
    self.len = len(os.listdir(videos_path))

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    batch_samples = self.samples[idx * self.batch_size:(idx+1)*self.batch_size]
    X_train = []
    y_train = []
    for batch_sample in batch_samples:      
      vidcap = cv2.VideoCapture(os.path.join(self.videos_path,batch_sample))              
      frames = []
      suc, frame = vidcap.read()              
      while suc: 
        frame = cv2.resize(frame, (self.width,self.height), interpolation=cv2.INTER_AREA)
        frame = mobilenet.preprocess_input(frame)   
        frames.append(frame)
        suc, frame = vidcap.read()
      
      X_train.append(frames)        
      cat = int(batch_sample.split('_')[0])#  Los mp4 estan categorizados de 1 a 64 y los .avi de 0 a 64, asi que no necesito restar 1        
      y_train.append(to_categorical(cat, num_classes=64))      
        
    X_train = np.asarray(X_train, dtype=np.float32)      
    y_train = np.asarray(y_train)      
    return X_train, y_train

INICIALIZACION


In [ ]:

def changeTestName(test_names):
  for i in range(len(test_names)):
    test_names_cleaned = list(map(lambda sample: "_".join(sample.split('_')[:-1])+".mp4", test_names))
  return list(set(test_names_cleaned))

def changeTestNameSimplesampling(test_names):
  for i in range(len(test_names)):
    test_names_cleaned = list(map(lambda sample: sample.split('.')[0]+".mp4", test_names))
  return list(set(test_names_cleaned))
"""
print(os.listdir("/content/gdrive/My Drive/Informática/Tesina/Experimentos/MobileNet+Conv"))
print("Ingrese el nombre del experimento: ")
exp_name = input()
if not os.path.exists(os.path.join("/content/gdrive/My Drive/Informática/Tesina/Experimentos/MobileNet+Conv", exp_name)):
  os.mkdir(os.path.join("/content/gdrive/My Drive/Informática/Tesina/Experimentos/MobileNet+Conv", exp_name))
print(os.listdir(os.path.join("/content/gdrive/My Drive/Informática/Tesina/Experimentos/MobileNet+Conv", exp_name)))
print("Nombre de modelo [Baseline]: ")
model_name = input()
model_name = "baseline" if model_name == '' else model_name
"""
experiment = Experiment(DATASET, MODEL_NAME, FOLD_NAME)
experiment.save_resume()
weights_path = experiment.weights_path
pickles_path = experiment.pickles_path
videos_path = experiment.videos_path



TRAIN VAL TEST SPLIT


In [ ]:

with open(os.path.join(experiment.names_path,f'train_{experiment.fold_name}_{experiment.files_posfix}.txt'), 'r') as f:
  train = f.read().split('\n')
with open(os.path.join(experiment.names_path,f'val_{experiment.fold_name}_{experiment.files_posfix}.txt'), 'r') as f:
  val = f.read().split('\n')
with open(os.path.join(experiment.names_path,f'test_{experiment.fold_name}_{experiment.files_posfix}.txt'), 'r') as f:
  test = f.read().split('\n')



In [ ]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    try:
        from keras import backend as K
    except:
        from tensorflow.keras import backend as K

    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        out_shape = l.output_shape
        if type(out_shape) is list:
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(p) for p in model.non_trainable_weights])

    number_size = 4.0
    if K.floatx() == 'float16':
        number_size = 2.0
    if K.floatx() == 'float64':
        number_size = 8.0

    total_memory = number_size * (batch_size * shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes

In [ ]:
print(get_model_memory_usage(32, model))

NameError: ignored

In [ ]:
print(get_model_memory_usage(32, model))

0.019


ENTRENAMIENTO

In [ ]:

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

batch_size = experiment.batch_size


# NO GUARDO MODELO POR AHORA

if f"{experiment.model_name}.pickle" in os.listdir(experiment.exp_folder):  
  with open(os.path.join(experiment.exp_folder, f"{experiment.model_name}.pickle"), 'rb') as pklfile:
    conf = pickle.load(pklfile)    
    model = Model.from_config(conf)
else:  
  model = MobileNet_model(experiment)
  #with open(os.path.join(experiment.exp_folder, f"{experiment.model_name}.pickle"), 'wb') as pklfile:
  #  conf = model.get_config()
  #  pickle.dump(conf, pklfile)
model.summary()

with open(os.path.join(experiment.exp_folder, f'{experiment.model_name}.txt'), 'w') as f:
  model.summary(print_fn=lambda x: f.write(x + '\n'))

if '{}_epoch_{}.h5'.format(experiment.model_name, experiment.current_epoch) in os.listdir(weights_path):
  model.load_weights(os.path.join(experiment.weights_path,'{}_epoch_{}.h5'.format(experiment.model_name, experiment.current_epoch)))


adam = Adam(learning_rate=experiment.lr)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

if experiment.testing:
  print("Estas seguro que queres testear? [y/n]")
  if (input() == 'y'):    
    eval_dict = model.evaluate(
        generator(videos_path, test, 1)
      , steps=int(len(test)/1)
      , batch_size=1
      , verbose = 1
      , return_dict=True
    )
    with open(os.path.join(experiment.exp_folder, f'evaluation_epoch{experiment.current_epoch}_toLength.pickle'), 'wb') as pkl:
      pickle.dump(eval_dict, pkl)
  else: 
    print("Cambia el experimento entonces en experiment.testing")
else:
  train_gen = SequenceGenerator(videos_path, train, batch_size, experiment.width, experiment.height, experiment.channels)    
  val_gen = SequenceGenerator(videos_path, val, batch_size, experiment.width, experiment.height, experiment.channels)    
  history = model.fit(
    train_gen
    , steps_per_epoch=int(len(train)/batch_size)
    , validation_data = val_gen
    ,validation_steps = int(len(val)/batch_size)
    , epochs = experiment.epochs_to_train
    , verbose = 1
    , shuffle = False    
    , callbacks=list(experiment.callbacks)
    , use_multiprocessing=True
      ,workers=4
    )


Model: "rgb128_2fold"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 128, 128, 3 0         
_________________________________________________________________
time_distributed (TimeDistri (None, None, 1024)        3228864   
_________________________________________________________________
gru (GRU)                    (None, None, 128)         443136    
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
Total params: 3,795,840
Trainable params: 566,976
Non-trainable params: 3,228,864
______________________________________

CROSS LENGTH TESTING


In [ ]:
import cv2
import os
import numpy as np
from keras.utils import to_categorical
from keras.applications import mobilenet
from keras.utils import Sequence
import math 
class SequenceGenerator(Sequence):
  def __init__(self, videos_path, samples,batch_size, width, height, channels):
    self.batch_size = batch_size
    self.videos_path = videos_path
    self.samples = samples
    self.width = width
    self.height = height
    self.channels = channels
    self.len = len(os.listdir(videos_path))

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    batch_samples = self.samples[idx * self.batch_size:(idx+1)*self.batch_size]
    X_train = []
    y_train = []
    for batch_sample in batch_samples:      
      vidcap = cv2.VideoCapture(os.path.join(self.videos_path,batch_sample))              
      frames = []
      suc, frame = vidcap.read()              
      while suc: 
        frame = cv2.resize(frame, (self.width,self.height), interpolation=cv2.INTER_AREA)
        frame = mobilenet.preprocess_input(frame)   
        frames.append(frame)
        suc, frame = vidcap.read()
      
      X_train.append(frames)        
      cat = int(batch_sample.split('_')[0])#  Los mp4 estan categorizados de 1 a 64 y los .avi de 0 a 64, asi que no necesito restar 1        
      y_train.append(to_categorical(cat, num_classes=64))      
        
    X_train = np.asarray(X_train, dtype=np.float32)      
    y_train = np.asarray(y_train)      
    return X_train, y_train




In [ ]:
DATASET = 'SimpleSampling32'
import numpy as np
import pickle
import functools as ft
from keras import Model
from keras.optimizers import Adam
def addHistories(h1, h2):    
    for key in ["accuracy", "loss", "val_loss", "val_accuracy"]:
        h2[key].append(h1[key]) #ya no es una lista asi que puedo acceder a la metrica directo 
    return h2

basePath = "/content/gdrive/My Drive/Informática/Tesina/"
baseFolder = basePath+'Experimentos/MobileNet+Conv/'+DATASET
namesPath = basePath+'DatasetsNames/'+DATASET

best_epochs = {}
mean_acc = {'8':[],'16':[], '32':[] }
videos_path = {'8':basePath+'Videos/Multisampling8/','16':basePath+'Videos/Multisampling16/', '32':basePath+'Videos/SimpleSampling32/' }

In [ ]:

for fold in ['2fold','3fold','4fold']:
  modelName = 'rgb'+fold
  expFolder = os.path.join(baseFolder,modelName)
  if f"{modelName}.pickle" in os.listdir(expFolder):  
    with open(os.path.join(expFolder, f"{modelName}.pickle"), 'rb') as pklfile:
      conf = pickle.load(pklfile)    
      model = Model.from_config(conf)
  else:
    print("no encuentro modelo")
  adam = Adam(learning_rate=0.001)
  model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

  EXP_PKL_PATH = os.path.join(expFolder, 'Pickles')  
  pkl_files = os.listdir(EXP_PKL_PATH)
  fls_s = sorted(pkl_files, key=lambda x: int(x.split('_')[-1].split('.')[0])) #Exp_18_epoch_2.pickle

  pkls = []
  for pkl_file in fls_s:
      with open(os.path.join(EXP_PKL_PATH ,pkl_file), 'rb') as pkl:
          h_temp = pickle.load(pkl)        
          print(h_temp)
          if 'val_loss' in h_temp.keys():
            pkls.append(h_temp)


  init_dict = {}
  for k in ["accuracy", "loss", "val_loss", "val_accuracy"]:
    init_dict[k] = []
  hs = ft.reduce(lambda a,b: addHistories(b,a), pkls, init_dict)
  best_epoch = np.array(hs['val_accuracy']).argmax()+1
  best_epochs[fold] = (best_epoch,hs['val_accuracy'][best_epoch-1])
  model.load_weights(os.path.join(expFolder,'Weights','{}_epoch_{}.h5'.format(modelName, best_epoch)))
  print(hs['val_accuracy'])
  print(best_epoch)
  with open(os.path.join(namesPath,f'test_{fold}_{DATASET}.txt'), 'r') as f:
    test = f.read().split('\n')  
  
  for num_frames in ['32']:
    path = videos_path[num_frames]
    gen = SequenceGenerator(path, test, 32, 64, 64, 3)    
    eval_dict = model.evaluate(
          gen
        , steps=int(len(test)/32)
        , batch_size=32
        , verbose = 1
        , return_dict=True
      )
    mean_acc[num_frames].append(eval_dict['accuracy'])
    with open(os.path.join(expFolder, f'evaluation_epoch{best_epoch}_{num_frames}frames.pickle'), 'wb') as pkl:
      pickle.dump(eval_dict, pkl)    
  
with open(os.path.join(baseFolder, f'tests.pickle'), 'wb') as pkl:
      pickle.dump(mean_acc, pkl)

with open(os.path.join(baseFolder, f'bestEpochs.pickle'), 'wb') as pkl:
      pickle.dump(best_epochs, pkl)        

{'loss': 3.027933120727539, 'accuracy': 0.20928029716014862, 'val_loss': 1.8524188995361328, 'val_accuracy': 0.4285714328289032}
{'loss': 1.2071391344070435, 'accuracy': 0.6169507503509521, 'val_loss': 0.9018262028694153, 'val_accuracy': 0.7232142686843872}
{'loss': 0.5611450672149658, 'accuracy': 0.8271780014038086, 'val_loss': 0.6210358738899231, 'val_accuracy': 0.796875}
{'loss': 0.30858346819877625, 'accuracy': 0.9100378751754761, 'val_loss': 0.43679705262184143, 'val_accuracy': 0.8616071343421936}
{'loss': 0.21335728466510773, 'accuracy': 0.9346590638160706, 'val_loss': 0.421249121427536, 'val_accuracy': 0.8526785969734192}
{'loss': 0.15428322553634644, 'accuracy': 0.9488636255264282, 'val_loss': 0.3881329596042633, 'val_accuracy': 0.875}
{'loss': 0.09316202998161316, 'accuracy': 0.9753788113594055, 'val_loss': 0.3241279125213623, 'val_accuracy': 0.9040178656578064}
{'loss': 0.061451997607946396, 'accuracy': 0.9796401262283325, 'val_loss': 0.3362032473087311, 'val_accuracy': 0.912

In [ ]:
os.listdir(EXP_PKL_PATH)

['rgb2fold_epoch_1.pickle',
 'rgb2fold_epoch_2.pickle',
 'rgb2fold_epoch_3.pickle',
 'rgb2fold_epoch_4.pickle',
 'rgb2fold_epoch_5.pickle',
 'rgb2fold_epoch_6.pickle',
 'rgb2fold_epoch_7.pickle',
 'rgb2fold_epoch_8.pickle',
 'rgb2fold_epoch_9.pickle',
 'rgb2fold_epoch_10.pickle',
 'rgb2fold_epoch_11.pickle',
 'rgb2fold_epoch_12.pickle',
 'rgb2fold_epoch_13.pickle',
 'rgb2fold_epoch_14.pickle',
 'rgb2fold_epoch_15.pickle',
 'rgb2fold_epoch_16.pickle',
 'rgb2fold_epoch_17.pickle',
 'rgb2fold_epoch_18.pickle',
 'rgb2fold_epoch_19.pickle',
 'rgb2fold_epoch_20.pickle',
 'rgb2fold_epoch_21.pickle',
 'rgb2fold_epoch_22.pickle',
 'rgb2fold_epoch_23.pickle',
 'rgb2fold_epoch_24.pickle',
 'rgb2fold_epoch_25.pickle',
 'rgb2fold_epoch_26.pickle',
 'rgb2fold_epoch_27.pickle',
 'rgb2fold_epoch_28 (1).pickle',
 'rgb2fold_epoch_29.pickle',
 'rgb2fold_epoch_30.pickle',
 'rgb2fold_epoch_31.pickle',
 'rgb2fold_epoch_32.pickle',
 'rgb2fold_epoch_33.pickle',
 'rgb2fold_epoch_34.pickle',
 'rgb2fold_epoch_35

In [ ]:
os.listdir(EXP_PKL_PATH)

['rgb2fold_epoch_1.pickle',
 'rgb2fold_epoch_2.pickle',
 'rgb2fold_epoch_3.pickle',
 'rgb2fold_epoch_4.pickle',
 'rgb2fold_epoch_5.pickle',
 'rgb2fold_epoch_6.pickle',
 'rgb2fold_epoch_7.pickle',
 'rgb2fold_epoch_8.pickle',
 'rgb2fold_epoch_9.pickle',
 'rgb2fold_epoch_10.pickle',
 'rgb2fold_epoch_11.pickle',
 'rgb2fold_epoch_12.pickle',
 'rgb2fold_epoch_13.pickle',
 'rgb2fold_epoch_14.pickle',
 'rgb2fold_epoch_15.pickle',
 'rgb2fold_epoch_16.pickle',
 'rgb2fold_epoch_17.pickle',
 'rgb2fold_epoch_18.pickle',
 'rgb2fold_epoch_19.pickle',
 'rgb2fold_epoch_20.pickle',
 'rgb2fold_epoch_21.pickle',
 'rgb2fold_epoch_22.pickle',
 'rgb2fold_epoch_23.pickle',
 'rgb2fold_epoch_24.pickle',
 'rgb2fold_epoch_25.pickle',
 'rgb2fold_epoch_26.pickle',
 'rgb2fold_epoch_27.pickle',
 'rgb2fold_epoch_28.pickle',
 'rgb2fold_epoch_29.pickle',
 'rgb2fold_epoch_30.pickle',
 'rgb2fold_epoch_31.pickle',
 'rgb2fold_epoch_32.pickle',
 'rgb2fold_epoch_33.pickle',
 'rgb2fold_epoch_34.pickle',
 'rgb2fold_epoch_35.pic